In [3]:
corpus = ['a', 'b', 'c', 'd', 'e']

In [4]:
def word_to_id(corpus):
    word_to_id = {}
    for idx, word in enumerate(corpus):
        word_to_id[word] = idx
    return word_to_id

In [6]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

# word_to_id 함수는 이미 구현되어 있음
def word_to_id(corpus):
    word_to_id = {}
    for idx, word in enumerate(corpus):
        word_to_id[word] = idx
    return word_to_id

class IngredientEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        """
        화장품 성분 임베딩 클래스
        Args:
            vocab_size: 어휘 크기 (성분 개수)
            embedding_dim: 임베딩 차원
        """
        super(IngredientEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding_dim = embedding_dim
        
    def forward(self, x):
        """
        입력 텐서를 임베딩 벡터로 변환
        Args:
            x: 입력 텐서 (batch_size, sequence_length)
        Returns:
            임베딩 벡터 (batch_size, sequence_length, embedding_dim)
        """
        return self.embedding(x)

def positional_encoding(seq_length, embedding_dim):
    """
    Positional encoding 생성
    Args:
        seq_length: 시퀀스 길이
        embedding_dim: 임베딩 차원
    Returns:
        positional encoding (seq_length, embedding_dim)
    """
    # positional encoding matrix 초기화
    pos_encoding = np.zeros((seq_length, embedding_dim))
    
    # position과 dimension 변수
    positions = np.arange(seq_length)[:, np.newaxis]
    dimensions = np.arange(embedding_dim)[np.newaxis, :]
    
    # 짝수 인덱스에는 sine 함수 적용, 홀수 인덱스에는 cosine 함수 적용
    angle_rates = 1 / np.power(10000, (2 * (dimensions // 2)) / embedding_dim)
    angle_rads = positions * angle_rates
    
    # 짝수 인덱스에 sine 적용
    pos_encoding[:, 0::2] = np.sin(angle_rads[:, 0::2])
    
    # 홀수 인덱스에 cosine 적용
    pos_encoding[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
    return torch.FloatTensor(pos_encoding)

class IngredientEmbeddingWithPositionalEncoding(nn.Module):
    def __init__(self, vocab_size, embedding_dim, max_seq_length):
        """
        화장품 성분 임베딩과 positional encoding을 결합한 클래스
        Args:
            vocab_size: 어휘 크기 (성분 개수)
            embedding_dim: 임베딩 차원
            max_seq_length: 최대 시퀀스 길이
        """
        super(IngredientEmbeddingWithPositionalEncoding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding_dim = embedding_dim
        self.max_seq_length = max_seq_length
        
        # positional encoding 생성
        self.pos_encoding = positional_encoding(max_seq_length, embedding_dim)
        
    def forward(self, x):
        """
        입력 텐서에 대해 임베딩과 positional encoding을 적용
        Args:
            x: 입력 텐서 (batch_size, sequence_length)
        Returns:
            임베딩 + positional encoding 벡터 (batch_size, sequence_length, embedding_dim)
        """
        # 임베딩 계산
        embeddings = self.embedding(x)
        
        # 시퀀스 길이 확인
        seq_length = x.size(1)
        
        # positional encoding 적용 (시퀀스 길이에 맞게 자름)
        positional_encodings = self.pos_encoding[:seq_length, :].unsqueeze(0)
        
        # 임베딩과 positional encoding 결합
        return embeddings + positional_encodings


